In [ ]:
#run only once
%pip install openai chromadb pandas numpy scikit-learn python-dotenv

In [ ]:
# ── Cell 2: Imports ─────────────────────────────────────────
import sys
sys.path.append('..')

from src.data_loader  import load_episodes
from src.vector_store import build_collection, get_collection
from src.search       import semantic_search
from src.llm_integeration import interpret_emotional_query
from src.memory       import ConversationMemory, Turn
from src.rag_pipeline import run_pipeline, print_results

print('✓ All imports OK')

In [ ]:
# ── Cell 0A: Install (once) ──────────────────────────────────
# %pip install youtube-transcript-api

# ── Cell 0B: Fetch all transcripts ──────────────────────────
import sys
sys.path.append('..')

from src.transcript_fetcher import fetch_all_transcripts, print_summary

# Reads URLs from data/youtube_urls.txt
# Saves results to data/processed/transcripts_df.csv
# Logs errors to logs/transcript_errors.log
df = fetch_all_transcripts(sleep_secs=1.0)

print_summary(df)

# ── Cell 0C: Pass URLs directly (alternative) ────────────────
my_urls = [
    "https://www.youtube.com/watch?v=TbsRU-crgsc",
    "https://www.youtube.com/watch?v=jroF3PH-PTs",
]

df = fetch_all_transcripts(urls=my_urls)
print_summary(df)



In [ ]:
# ── Cell 3: Load data ────────────────────────────────────────
df = load_episodes()
df[['youtube_title', 'youtube_channel', 'duration_mins', 'word_count']].head()

In [ ]:
# ── Cell 4: Build ChromaDB (RUN ONCE ONLY) ──────────────────
collection = build_collection(df, force_rebuild=False)
print(f'Collection has {collection.count()} episodes')

In [ ]:
# ── Cell 5: Load existing collection (USE EVERY SESSION) ────
collection = get_collection()

In [ ]:
# ── Cell 6: Test search ──────────────────────────────────────
test_queries = [
    'I feel like a failure after not getting that job',
    'I am angry and I do not know why',
    'I want to love myself more',
]

for q in test_queries:
    print(f"\n🔍 '{q}'")
    results = semantic_search(q, collection, top_k=2)
    for r in results:
        print(f"  → {r['metadata']['episode_title'][:55]}  ({r['similarity']:.2f})")

In [ ]:
# ── Cell 7: Full RAG pipeline ────────────────────────────────
memory     = ConversationMemory()
user_query = 'I feel really stressed about finding a new job and keep doubting myself'

output = run_pipeline(
    user_query = user_query,
    collection = collection,
    memory     = memory,
    top_k      = 3,
)

print_results(output)

In [ ]:
# ── Cell 8: Check memory ─────────────────────────────────────
print(f'Memory has {len(memory)} turn(s)\n')
print(memory.build_context_string())

In [ ]:
# ── Cell 9: Second turn (memory feeds into LLM) ─────────────
output2 = run_pipeline(
    user_query = 'Now I feel ashamed that I am not handling this better',
    collection = collection,
    memory     = memory,
    top_k      = 2,
)

print_results(output2)
print(f'\nMemory now has {len(memory)} turns')

In [ ]:
# ── Cell 10: Reset memory ────────────────────────────────────
memory.clear()
print(f'Cleared. Turns: {len(memory)}')